In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import multiprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from tqdm import notebook
import cv2 as cv
import pickle

from lshash import LSHash

import torch
from torchvision import models, transforms#, utils
import torch.nn as nn 
from torch.optim import Adam

import adabound as adabound

###local scripts
from geographic_zones import create_geo_zones, classify_geographic_zones
from Custom_dataloader_geozones import MapDataset, Rescale, CenterCrop, Normalize, ToTensor

# ResNet18

In [ ]:
if __name__ == '__main__':
    
    startTime = time.time()
        
#     # define a seed for reproducibility
#     seed = 5436457
#     torch.manual_seed(seed)
    
    ##============Create geographic zones====================
    ###Training set
    trainDF = pd.read_csv("./data/piom_train_30k.csv")
    trainDF.drop(columns=["Unnamed: 0"],inplace=True)
    lon_dim = 4.
    lat_dim = 2.
    Geo_zones = create_geo_zones(list(trainDF['llcrnrlon']), list(trainDF['llcrnrlat']), lon_dim, lat_dim)
    n_zones = len(Geo_zones)
    print('Number of geographic zones: ', n_zones)
    trainDF['Geo_zone'] = trainDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)    
#     ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(trainDF['llcrnrlon'], trainDF['llcrnrlat'], hue=trainDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", n_zones), legend=False)
#     plt.title('Training samples per Geographic Zones')
#     plt.show()
#     ##Visualizing distribution of the geographic zones
#     plt.figure(figsize=(15, 4))
#     trainDF['Geo_zone'].value_counts().plot(kind='bar')
#     plt.ylabel('Count')
#     plt.xlabel('Label')
#     plt.title('Sample distribution per Geographic Zones')
#     plt.show()
    
    ###Validation set
    valDF = pd.read_csv("./data/piom_train2_10k.csv")
    valDF.drop(columns=["Unnamed: 0"],inplace=True)
    valDF['Geo_zone'] = valDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)
    ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(valDF['llcrnrlon'], valDF['llcrnrlat'], hue=valDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", len(valDF['Geo_zone'].unique())), legend=False)
#     plt.title('Validation samples per Geographic Zones')
#     plt.show()
    
    ##============Create dataloader====================
    ##Training dataloader
    train_batch_size = 50
    train_map_data = MapDataset(trainDF, './data/piom_train_png_30k/', transform= transforms.Compose([
                                                Rescale(225),
                                                CenterCrop((224,224)),
                                                Normalize(alpha=0., beta=1.),
                                                ToTensor(),
                                            ]))
    train_loader = torch.utils.data.DataLoader(train_map_data,
                                            batch_size=train_batch_size,
                                            shuffle=True,
                                            num_workers=multiprocessing.cpu_count())
    ##Validation dataloader
    val_batch_size = 50
    validation_map_data = MapDataset(valDF, './data/piom_train2_png_10k/', transform= transforms.Compose([
                                                   Rescale(225),
                                                   CenterCrop((224,224)),
                                                   Normalize(alpha=0., beta=1.),
                                                   ToTensor(),
                                               ]))
    validation_loader = torch.utils.data.DataLoader(validation_map_data,
                                              batch_size=val_batch_size,
                                              shuffle=True,
                                              num_workers=multiprocessing.cpu_count(),
                                              drop_last=False)
    print('Training batch size: ', train_batch_size)
    print('Validation batch size: ', val_batch_size)
    
#     ##try data loader
#     for i_batch, sample_batch in enumerate(train_loader):
#         print(i_batch, sample_batch['image'].size())
#         if i_batch == 1:
#             break
            
    ##=============Transfer Learning from ResNet34 model for classification==========
    model = models.resnet18(pretrained=True, progress=True)
#     print(model)
    ##modify last fully connected layer
    model.fc = nn.Linear(512,n_zones) #resnet18:512,resnet34:512, resnet34:2048, resnext:2048,
    optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
    ###defining the loss function
    criterion = nn.CrossEntropyLoss()
    ###model on GPU if GPU is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = criterion.to(device)
    #print(model)
    
    ##============Train the model on geographic zone classification - only the last FC layer====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 10:
            for param in child.parameters():
                param.requires_grad = False
    n_epochs = 1
    print('============ Training phase 1 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    ##============Train the model on geographic zone classification - all layers====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 10:
            for param in child.parameters():
                param.requires_grad = True
    n_epochs = 5
    print('============ Training phase 2 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    
#     ##========Drop last FC layer of the model for image embedding=======
#     new_classifier = nn.Sequential(*list(model.children())[:-1])
    
#     ##=================save model in pth file==========================
#     MODEL_PATH = './models/gpu_model_resnet18_4_25.pth'
#     torch.save(new_classifier.state_dict(), MODEL_PATH)
#     print('Model saved!!')
    print("Training, time: {}h {}min {}s".format((time.time()-startTime)//3600, 
                                                                      ((time.time()-startTime)%3600)//60,
                                                                      (time.time()-startTime)%60))

# VGG11

In [2]:
if __name__ == '__main__':
    
    startTime = time.time()
        
#     # define a seed for reproducibility
#     seed = 5436457
#     torch.manual_seed(seed)
    
    ##============Create geographic zones====================
    ###Training set
    trainDF = pd.read_csv("./data/piom_train_30k.csv")
    trainDF.drop(columns=["Unnamed: 0"],inplace=True)
    lon_dim = 4.
    lat_dim = 2.
    Geo_zones = create_geo_zones(list(trainDF['llcrnrlon']), list(trainDF['llcrnrlat']), lon_dim, lat_dim)
    n_zones = len(Geo_zones)
    print('Number of geographic zones: ', n_zones)
    trainDF['Geo_zone'] = trainDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)    
#     ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(trainDF['llcrnrlon'], trainDF['llcrnrlat'], hue=trainDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", n_zones), legend=False)
#     plt.title('Training samples per Geographic Zones')
#     plt.show()
#     ##Visualizing distribution of the geographic zones
#     plt.figure(figsize=(15, 4))
#     trainDF['Geo_zone'].value_counts().plot(kind='bar')
#     plt.ylabel('Count')
#     plt.xlabel('Label')
#     plt.title('Sample distribution per Geographic Zones')
#     plt.show()
    
    ###Validation set
    valDF = pd.read_csv("./data/piom_train2_10k.csv")
    valDF.drop(columns=["Unnamed: 0"],inplace=True)
    valDF['Geo_zone'] = valDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)
    ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(valDF['llcrnrlon'], valDF['llcrnrlat'], hue=valDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", len(valDF['Geo_zone'].unique())), legend=False)
#     plt.title('Validation samples per Geographic Zones')
#     plt.show()
    
    ##============Create dataloader====================
    ##Training dataloader
    train_batch_size = 50
    train_map_data = MapDataset(trainDF, './data/piom_train_png_30k/', transform= transforms.Compose([
                                                Rescale(225),
                                                CenterCrop((224,224)),
                                                Normalize(alpha=0., beta=1.),
                                                ToTensor(),
                                            ]))
    train_loader = torch.utils.data.DataLoader(train_map_data,
                                            batch_size=train_batch_size,
                                            shuffle=True,
                                            num_workers=multiprocessing.cpu_count())
    ##Validation dataloader
    val_batch_size = 50
    validation_map_data = MapDataset(valDF, './data/piom_train2_png_10k/', transform= transforms.Compose([
                                                   Rescale(225),
                                                   CenterCrop((224,224)),
                                                   Normalize(alpha=0., beta=1.),
                                                   ToTensor(),
                                               ]))
    validation_loader = torch.utils.data.DataLoader(validation_map_data,
                                              batch_size=val_batch_size,
                                              shuffle=True,
                                              num_workers=multiprocessing.cpu_count(),
                                              drop_last=False)
    print('Training batch size: ', train_batch_size)
    print('Validation batch size: ', val_batch_size)
    
#     ##try data loader
#     for i_batch, sample_batch in enumerate(train_loader):
#         print(i_batch, sample_batch['image'].size())
#         if i_batch == 1:
#             break
            
    ##=============Transfer Learning from ResNet34 model for classification==========
    model = models.vgg11(pretrained=True, progress=True)
    ##modify last fully connected layer
#     print(model)
    model.fc = nn.Linear(1000,n_zones) #resnet18:512,resnet34:512, resnet34:2048, resnext:2048,
    optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
    ###defining the loss function
    criterion = nn.CrossEntropyLoss()
    ###model on GPU if GPU is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = criterion.to(device)
    #print(model)
    
    ##============Train the model on geographic zone classification - only the last FC layer====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 3:
            for param in child.parameters():
                param.requires_grad = False
    n_epochs = 1
    print('============ Training phase 1 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    ##============Train the model on geographic zone classification - all layers====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 3:
            for param in child.parameters():
                param.requires_grad = True
    n_epochs = 5
    print('============ Training phase 2 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    
#     ##========Drop last FC layer of the model for image embedding=======
#     new_classifier = nn.Sequential(*list(model.children())[:-1])
    
#     ##=================save model in pth file==========================
#     MODEL_PATH = './models/gpu_model_resnet18_4_25.pth'
#     torch.save(new_classifier.state_dict(), MODEL_PATH)
#     print('Model saved!!')
    print("Training, time: {}h {}min {}s".format((time.time()-startTime)//3600, 
                                                                      ((time.time()-startTime)%3600)//60,
                                                                      (time.time()-startTime)%60))

Number of geographic zones:  676
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Training batch size:  50
Validation batch size:  50
============ Training phase 1 - number of epochs: 1 ============


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 2.52 GiB already allocated; 81.50 MiB free; 2.92 GiB reserved in total by PyTorch)

# VGG11_bn

In [ ]:
if __name__ == '__main__':
    
    startTime = time.time()
        
#     # define a seed for reproducibility
#     seed = 5436457
#     torch.manual_seed(seed)
    
    ##============Create geographic zones====================
    ###Training set
    trainDF = pd.read_csv("./data/piom_train_30k.csv")
    trainDF.drop(columns=["Unnamed: 0"],inplace=True)
    lon_dim = 4.
    lat_dim = 2.
    Geo_zones = create_geo_zones(list(trainDF['llcrnrlon']), list(trainDF['llcrnrlat']), lon_dim, lat_dim)
    n_zones = len(Geo_zones)
    print('Number of geographic zones: ', n_zones)
    trainDF['Geo_zone'] = trainDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)    
#     ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(trainDF['llcrnrlon'], trainDF['llcrnrlat'], hue=trainDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", n_zones), legend=False)
#     plt.title('Training samples per Geographic Zones')
#     plt.show()
#     ##Visualizing distribution of the geographic zones
#     plt.figure(figsize=(15, 4))
#     trainDF['Geo_zone'].value_counts().plot(kind='bar')
#     plt.ylabel('Count')
#     plt.xlabel('Label')
#     plt.title('Sample distribution per Geographic Zones')
#     plt.show()
    
    ###Validation set
    valDF = pd.read_csv("./data/piom_train2_10k.csv")
    valDF.drop(columns=["Unnamed: 0"],inplace=True)
    valDF['Geo_zone'] = valDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)
    ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(valDF['llcrnrlon'], valDF['llcrnrlat'], hue=valDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", len(valDF['Geo_zone'].unique())), legend=False)
#     plt.title('Validation samples per Geographic Zones')
#     plt.show()
    
    ##============Create dataloader====================
    ##Training dataloader
    train_batch_size = 50
    train_map_data = MapDataset(trainDF, './data/piom_train_png_30k/', transform= transforms.Compose([
                                                Rescale(225),
                                                CenterCrop((224,224)),
                                                Normalize(alpha=0., beta=1.),
                                                ToTensor(),
                                            ]))
    train_loader = torch.utils.data.DataLoader(train_map_data,
                                            batch_size=train_batch_size,
                                            shuffle=True,
                                            num_workers=multiprocessing.cpu_count())
    ##Validation dataloader
    val_batch_size = 50
    validation_map_data = MapDataset(valDF, './data/piom_train2_png_10k/', transform= transforms.Compose([
                                                   Rescale(225),
                                                   CenterCrop((224,224)),
                                                   Normalize(alpha=0., beta=1.),
                                                   ToTensor(),
                                               ]))
    validation_loader = torch.utils.data.DataLoader(validation_map_data,
                                              batch_size=val_batch_size,
                                              shuffle=True,
                                              num_workers=multiprocessing.cpu_count(),
                                              drop_last=False)
    print('Training batch size: ', train_batch_size)
    print('Validation batch size: ', val_batch_size)
    
#     ##try data loader
#     for i_batch, sample_batch in enumerate(train_loader):
#         print(i_batch, sample_batch['image'].size())
#         if i_batch == 1:
#             break
            
    ##=============Transfer Learning from ResNet34 model for classification==========
    model = models.vgg11_bn(pretrained=True, progress=True)
    ##modify last fully connected layer
#     print(model)
    model.fc = nn.Linear(1000,n_zones) #resnet18:512,resnet34:512, resnet34:2048, resnext:2048,
    optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
    ###defining the loss function
    criterion = nn.CrossEntropyLoss()
    ###model on GPU if GPU is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = criterion.to(device)
    #print(model)
    
    ##============Train the model on geographic zone classification - only the last FC layer====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 3:
            for param in child.parameters():
                param.requires_grad = False
    n_epochs = 1
    print('============ Training phase 1 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    ##============Train the model on geographic zone classification - all layers====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 3:
            for param in child.parameters():
                param.requires_grad = True
    n_epochs = 5
    print('============ Training phase 2 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    
#     ##========Drop last FC layer of the model for image embedding=======
#     new_classifier = nn.Sequential(*list(model.children())[:-1])
    
#     ##=================save model in pth file==========================
#     MODEL_PATH = './models/gpu_model_resnet18_4_25.pth'
#     torch.save(new_classifier.state_dict(), MODEL_PATH)
#     print('Model saved!!')
    print("Training, time: {}h {}min {}s".format((time.time()-startTime)//3600, 
                                                                      ((time.time()-startTime)%3600)//60,
                                                                      (time.time()-startTime)%60))

# mnasnet0_5

In [3]:
# if __name__ == '__main__':
    
#     startTime = time.time()
        
# #     # define a seed for reproducibility
# #     seed = 5436457
# #     torch.manual_seed(seed)
    
#     ##============Create geographic zones====================
#     ###Training set
#     trainDF = pd.read_csv("./data/piom_train_30k.csv")
#     trainDF.drop(columns=["Unnamed: 0"],inplace=True)
#     lon_dim = 4.
#     lat_dim = 2.
#     Geo_zones = create_geo_zones(list(trainDF['llcrnrlon']), list(trainDF['llcrnrlat']), lon_dim, lat_dim)
#     n_zones = len(Geo_zones)
#     print('Number of geographic zones: ', n_zones)
#     trainDF['Geo_zone'] = trainDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)    
# #     ## plot geographic zones
# #     plt.figure(figsize = (15,8))
# #     sns.scatterplot(trainDF['llcrnrlon'], trainDF['llcrnrlat'], hue=trainDF['Geo_zone'], 
# #                     palette=sns.color_palette("hls", n_zones), legend=False)
# #     plt.title('Training samples per Geographic Zones')
# #     plt.show()
# #     ##Visualizing distribution of the geographic zones
# #     plt.figure(figsize=(15, 4))
# #     trainDF['Geo_zone'].value_counts().plot(kind='bar')
# #     plt.ylabel('Count')
# #     plt.xlabel('Label')
# #     plt.title('Sample distribution per Geographic Zones')
# #     plt.show()
    
#     ###Validation set
#     valDF = pd.read_csv("./data/piom_train2_10k.csv")
#     valDF.drop(columns=["Unnamed: 0"],inplace=True)
#     valDF['Geo_zone'] = valDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)
#     ## plot geographic zones
# #     plt.figure(figsize = (15,8))
# #     sns.scatterplot(valDF['llcrnrlon'], valDF['llcrnrlat'], hue=valDF['Geo_zone'], 
# #                     palette=sns.color_palette("hls", len(valDF['Geo_zone'].unique())), legend=False)
# #     plt.title('Validation samples per Geographic Zones')
# #     plt.show()
    
#     ##============Create dataloader====================
#     ##Training dataloader
#     train_batch_size = 50
#     train_map_data = MapDataset(trainDF, './data/piom_train_png_30k/', transform= transforms.Compose([
#                                                 Rescale(225),
#                                                 CenterCrop((224,224)),
#                                                 Normalize(alpha=0., beta=1.),
#                                                 ToTensor(),
#                                             ]))
#     train_loader = torch.utils.data.DataLoader(train_map_data,
#                                             batch_size=train_batch_size,
#                                             shuffle=True,
#                                             num_workers=multiprocessing.cpu_count())
#     ##Validation dataloader
#     val_batch_size = 50
#     validation_map_data = MapDataset(valDF, './data/piom_train2_png_10k/', transform= transforms.Compose([
#                                                    Rescale(225),
#                                                    CenterCrop((224,224)),
#                                                    Normalize(alpha=0., beta=1.),
#                                                    ToTensor(),
#                                                ]))
#     validation_loader = torch.utils.data.DataLoader(validation_map_data,
#                                               batch_size=val_batch_size,
#                                               shuffle=True,
#                                               num_workers=multiprocessing.cpu_count(),
#                                               drop_last=False)
#     print('Training batch size: ', train_batch_size)
#     print('Validation batch size: ', val_batch_size)
    
# #     ##try data loader
# #     for i_batch, sample_batch in enumerate(train_loader):
# #         print(i_batch, sample_batch['image'].size())
# #         if i_batch == 1:
# #             break
            
#     ##=============Transfer Learning from ResNet34 model for classification==========
#     model = models.mnasnet0_5(pretrained=True, progress=True)
#     ##modify last fully connected layer
# #     print(model)
#     model.fc = nn.Linear(1000,n_zones) #resnet18:512,resnet34:512, resnet34:2048, resnext:2048,
#     optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
#     ###defining the loss function
#     criterion = nn.CrossEntropyLoss()
#     ###model on GPU if GPU is available
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
#     criterion = criterion.to(device)
#     #print(model)
    
#     ##============Train the model on geographic zone classification - only the last FC layer====================
#     ##freeze all layers but the last
#     ct = 0
#     for child in model.children():
#         ct += 1
#         if ct < 2:
#             for param in child.parameters():
#                 param.requires_grad = False
#     n_epochs = 1
#     print('============ Training phase 1 - number of epochs: {} ============'.format(n_epochs))
#     verbose = True
#     total_step = len(train_loader)
#     loss_list = []
#     accuracy_list = []
#     compute_validation = True
#     val_accuracy_list = []
#     for epoch in range(n_epochs):
#         for i_batch, sample_batch in enumerate(train_loader):
#             # Forward pass
#             train_X = sample_batch['image'].float().to(device)
#             train_Y = sample_batch['Geo_zone'].long().to(device)
#             outputs = model(train_X)
#             loss = criterion(outputs, train_Y)
#             loss_list.append(loss.item())

#             # Backprop and optimisation
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # Track the accuracy
#             softmax = torch.exp(outputs).cpu()
#             prob = list(softmax.detach().numpy())
#             predicted = np.argmax(prob, axis=1)
#             accuracy = accuracy_score(predicted,
#                                        train_Y.detach().cpu())
#             accuracy_list.append(accuracy)

#             if verbose:
#                 if (i_batch + 1) % 50 == 0:
#                     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
#                           .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
#         if compute_validation:
#             val_predicted = []
#             val_true = []
#             for i_batch, sample_batch in enumerate(validation_loader):
#                 # prediction for validation set
#                 with torch.no_grad():
#                     val_X = sample_batch['image'].float().to(device)
#                     val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
#                     outputs = model(val_X)
#                     softmax = torch.exp(outputs).cpu()
#                     prob = list(softmax.detach().numpy())
#                     predicted = np.argmax(prob, axis=1)
#                     val_predicted.extend(predicted)
#                 if i_batch == 9:
#                     break
#             val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
#             val_accuracy_list.append(val_accuracy)
#             print('Validation accuracy: {}'.format(val_accuracy))
#     ##============Train the model on geographic zone classification - all layers====================
#     ##freeze all layers but the last
#     ct = 0
#     for child in model.children():
#         ct += 1
#         if ct < 2:
#             for param in child.parameters():
#                 param.requires_grad = True
#     n_epochs = 5
#     print('============ Training phase 2 - number of epochs: {} ============'.format(n_epochs))
#     verbose = True
#     total_step = len(train_loader)
#     loss_list = []
#     accuracy_list = []
#     compute_validation = True
#     val_accuracy_list = []
#     for epoch in range(n_epochs):
#         for i_batch, sample_batch in enumerate(train_loader):
#             # Forward pass
#             train_X = sample_batch['image'].float().to(device)
#             train_Y = sample_batch['Geo_zone'].long().to(device)
#             outputs = model(train_X)
#             loss = criterion(outputs, train_Y)
#             loss_list.append(loss.item())

#             # Backprop and optimisation
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # Track the accuracy
#             softmax = torch.exp(outputs).cpu()
#             prob = list(softmax.detach().numpy())
#             predicted = np.argmax(prob, axis=1)
#             accuracy = accuracy_score(predicted,
#                                        train_Y.detach().cpu())
#             accuracy_list.append(accuracy)

#             if verbose:
#                 if (i_batch + 1) % 50 == 0:
#                     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
#                           .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
#         if compute_validation:
#             val_predicted = []
#             val_true = []
#             for i_batch, sample_batch in enumerate(validation_loader):
#                 # prediction for validation set
#                 with torch.no_grad():
#                     val_X = sample_batch['image'].float().to(device)
#                     val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
#                     outputs = model(val_X)
#                     softmax = torch.exp(outputs).cpu()
#                     prob = list(softmax.detach().numpy())
#                     predicted = np.argmax(prob, axis=1)
#                     val_predicted.extend(predicted)
#                 if i_batch == 9:
#                     break
#             val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
#             val_accuracy_list.append(val_accuracy)
#             print('Validation accuracy: {}'.format(val_accuracy))
    
# #     ##========Drop last FC layer of the model for image embedding=======
# #     new_classifier = nn.Sequential(*list(model.children())[:-1])
    
# #     ##=================save model in pth file==========================
# #     MODEL_PATH = './models/gpu_model_resnet18_4_25.pth'
# #     torch.save(new_classifier.state_dict(), MODEL_PATH)
# #     print('Model saved!!')
#     print("Training, time: {}h {}min {}s".format((time.time()-startTime)//3600, 
#                                                                       ((time.time()-startTime)%3600)//60,
#                                                                       (time.time()-startTime)%60))

In [4]:
model = models.shufflenet_v2_x0_5(pretrained=True, progress=True)
##modify last fully connected layer
# model.fc = nn.Linear(1000,12)
print(model)
ct = 0
for child in model.children():
    ct += 1
print('number of chidren: ', ct)

Downloading: "https://download.pytorch.org/models/shufflenetv2_x0.5-f707e7126e.pth" to C:\Users\qc16/.cache\torch\checkpoints\shufflenetv2_x0.5-f707e7126e.pth



ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running

# AlexNet

In [2]:
if __name__ == '__main__':
    
    startTime = time.time()
        
#     # define a seed for reproducibility
#     seed = 5436457
#     torch.manual_seed(seed)
    
    ##============Create geographic zones====================
    ###Training set
    trainDF = pd.read_csv("./data/piom_train_30k.csv")
    trainDF.drop(columns=["Unnamed: 0"],inplace=True)
    lon_dim = 4.
    lat_dim = 2.
    Geo_zones = create_geo_zones(list(trainDF['llcrnrlon']), list(trainDF['llcrnrlat']), lon_dim, lat_dim)
    n_zones = len(Geo_zones)
    print('Number of geographic zones: ', n_zones)
    trainDF['Geo_zone'] = trainDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)    
#     ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(trainDF['llcrnrlon'], trainDF['llcrnrlat'], hue=trainDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", n_zones), legend=False)
#     plt.title('Training samples per Geographic Zones')
#     plt.show()
#     ##Visualizing distribution of the geographic zones
#     plt.figure(figsize=(15, 4))
#     trainDF['Geo_zone'].value_counts().plot(kind='bar')
#     plt.ylabel('Count')
#     plt.xlabel('Label')
#     plt.title('Sample distribution per Geographic Zones')
#     plt.show()
    
    ###Validation set
    valDF = pd.read_csv("./data/piom_train2_10k.csv")
    valDF.drop(columns=["Unnamed: 0"],inplace=True)
    valDF['Geo_zone'] = valDF.apply(lambda row: classify_geographic_zones(Geo_zones, row, lon_dim, lat_dim), axis=1)
    ## plot geographic zones
#     plt.figure(figsize = (15,8))
#     sns.scatterplot(valDF['llcrnrlon'], valDF['llcrnrlat'], hue=valDF['Geo_zone'], 
#                     palette=sns.color_palette("hls", len(valDF['Geo_zone'].unique())), legend=False)
#     plt.title('Validation samples per Geographic Zones')
#     plt.show()
    
    ##============Create dataloader====================
    ##Training dataloader
    train_batch_size = 50
    train_map_data = MapDataset(trainDF, './data/piom_train_png_30k/', transform= transforms.Compose([
                                                Rescale(225),
                                                CenterCrop((224,224)),
                                                Normalize(alpha=0., beta=1.),
                                                ToTensor(),
                                            ]))
    train_loader = torch.utils.data.DataLoader(train_map_data,
                                            batch_size=train_batch_size,
                                            shuffle=True,
                                            num_workers=multiprocessing.cpu_count())
    ##Validation dataloader
    val_batch_size = 50
    validation_map_data = MapDataset(valDF, './data/piom_train2_png_10k/', transform= transforms.Compose([
                                                   Rescale(225),
                                                   CenterCrop((224,224)),
                                                   Normalize(alpha=0., beta=1.),
                                                   ToTensor(),
                                               ]))
    validation_loader = torch.utils.data.DataLoader(validation_map_data,
                                              batch_size=val_batch_size,
                                              shuffle=True,
                                              num_workers=multiprocessing.cpu_count(),
                                              drop_last=False)
    print('Training batch size: ', train_batch_size)
    print('Validation batch size: ', val_batch_size)
    
#     ##try data loader
#     for i_batch, sample_batch in enumerate(train_loader):
#         print(i_batch, sample_batch['image'].size())
#         if i_batch == 1:
#             break
            
    ##=============Transfer Learning from ResNet34 model for classification==========
    model = models.shufflenet_v2_x0_5(pretrained=True, progress=True)
    ##modify last fully connected layer
#     print(model)
#     model.classifier[1] = nn.Conv2d(512, n_zones, kernel_size=(1,1), stride=(1,1))
    model.fc = nn.Linear(1024,n_zones) #resnet18:512,resnet34:512, resnet34:2048, resnext:2048,
    # change the internal num_classes variable rather than redefining the forward pass
    model.num_classes = n_zones
    optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
    ###defining the loss function
    criterion = nn.CrossEntropyLoss()
    ###model on GPU if GPU is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = criterion.to(device)
    #print(model)
    
    ##============Train the model on geographic zone classification - only the last FC layer====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 7:
            for param in child.parameters():
                param.requires_grad = False
    n_epochs = 1
    print('============ Training phase 1 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    ##============Train the model on geographic zone classification - all layers====================
    ##freeze all layers but the last
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 7:
            for param in child.parameters():
                param.requires_grad = True
    n_epochs = 10
    print('============ Training phase 2 - number of epochs: {} ============'.format(n_epochs))
    verbose = True
    total_step = len(train_loader)
    loss_list = []
    accuracy_list = []
    compute_validation = True
    val_accuracy_list = []
    for epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
            # Forward pass
            train_X = sample_batch['image'].float().to(device)
            train_Y = sample_batch['Geo_zone'].long().to(device)
            outputs = model(train_X)
            loss = criterion(outputs, train_Y)
            loss_list.append(loss.item())

            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            softmax = torch.exp(outputs).cpu()
            prob = list(softmax.detach().numpy())
            predicted = np.argmax(prob, axis=1)
            accuracy = accuracy_score(predicted,
                                       train_Y.detach().cpu())
            accuracy_list.append(accuracy)

            if verbose:
                if (i_batch + 1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, n_epochs, i_batch + 1, total_step, loss.item(), accuracy))
        if compute_validation:
            val_predicted = []
            val_true = []
            for i_batch, sample_batch in enumerate(validation_loader):
                # prediction for validation set
                with torch.no_grad():
                    val_X = sample_batch['image'].float().to(device)
                    val_true.extend(sample_batch['Geo_zone'].long().detach().cpu())
                    outputs = model(val_X)
                    softmax = torch.exp(outputs).cpu()
                    prob = list(softmax.detach().numpy())
                    predicted = np.argmax(prob, axis=1)
                    val_predicted.extend(predicted)
                if i_batch == 9:
                    break
            val_accuracy = accuracy_score(val_predicted, torch.stack(val_true))
            val_accuracy_list.append(val_accuracy)
            print('Validation accuracy: {}'.format(val_accuracy))
    
#     ##========Drop last FC layer of the model for image embedding=======
#     new_classifier = nn.Sequential(*list(model.children())[:-1])
    
#     ##=================save model in pth file==========================
#     MODEL_PATH = './models/gpu_model_resnet18_4_25.pth'
#     torch.save(new_classifier.state_dict(), MODEL_PATH)
#     print('Model saved!!')
    print("Training, time: {}h {}min {}s".format((time.time()-startTime)//3600, 
                                                  ((time.time()-startTime)%3600)//60,
                                                  (time.time()-startTime)%60))

Number of geographic zones:  676
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Out of Zone
Training batch size:  50
Validation batch size:  50
============ Training phase 1 - number of epochs: 1 ============
Epoch [1/1], Step [50/600], Loss: 6.4327, Accuracy: 0.00%
Epoch [1/1], Step [100/600], Loss: 6.4193, Accuracy: 0.00%
Epoch [1/1], Step [150/600], Loss: 6.3251, Accuracy: 0.00%
Epoch [1/1], Step [200/600], Loss: 6.2765, Accuracy: 0.04%
Epoch [1/1], Step [250/600], Loss: 6.2504, Accuracy: 0.00%
Epoch [1/1], Step [300/600], Loss: 6.2451, Accuracy: 0.02%
Epoch [1/1], Step [350/600], Loss: 6.2710, Accuracy: 0.00%
Epoch [1/1], Step [400/600], Loss: 6.2186, Accuracy: 0.00%
Epoch [1/1], Step [450/600], Loss: 6.1874, Accuracy: 0.02%
Epoch [1/1], Step [500/600], Loss: 6.2688, Accuracy: 0.00%
Epoch [1/1], Step [550/600], Loss

KeyboardInterrupt: 